生成n元fcc高熵合金

In [29]:
import random
import numpy as np
import pandas as pd
from itertools import combinations
from tqdm import tqdm


# 元素性质字典（调整候选元素）
elements = {
    'Co': {'Tm': 1768.0, 'radius': 1.25, 'VEC': 9},
    'Cr': {'Tm': 2130.0, 'radius': 1.29, 'VEC': 6},
    'Fe': {'Tm': 1811.0, 'radius': 1.26, 'VEC': 8},
    'Mn': {'Tm': 1519.0, 'radius': 1.37, 'VEC': 7},
    'Ni': {'Tm': 1728.0, 'radius': 1.25, 'VEC': 10},
    'V': {'Tm': 2190.0, 'radius': 1.35, 'VEC': 5},
}

# 混合焓字典
mixen_dict = {
    'CoCr': -4.0, 'CoFe': -1.0, 'CoMn': -5.0, 'CoNi': 0.0, 'CoV': -14.0,
    'CrFe': -1.0, 'CrMn': 2.0, 'CrNi': -7.0, 'CrV': -2.0, 
    'FeMn': 0.0, 'FeNi': -2.0, 'FeV': -7.0, 
    'MnNi': -8.0, 'MnV': -1.0,
    'NiV': -18.0, 
}

# 元素固定顺序
elements_order = ['Co', 'Cr', 'Fe', 'Mn', 'Ni', 'V']

# 预计算存储
valid_compositions = None
total_count = 0

def validate_config():
    """验证配置参数有效性"""
    cfg = GLOBAL_CONFIG
    if cfg['min_percent'] > cfg['max_percent']:
        raise ValueError("最小浓度不能大于最大浓度")
    if 100 % cfg['step'] != 0:
        raise ValueError("步长必须能整除100")
    if cfg['num_elements'] < 1:
        raise ValueError("主元数必须≥1")
    if cfg['min_percent'] % cfg['step'] != 0 or cfg['max_percent'] % cfg['step'] != 0:
        raise ValueError("浓度边界必须能被步长整除")

def generate_concentrations():
    """动态生成所有合法浓度分配"""
    cfg = GLOBAL_CONFIG
    n = cfg['num_elements']
    min_unit = cfg['min_percent'] // cfg['step']
    max_unit = cfg['max_percent'] // cfg['step']
    total_units = 100 // cfg['step']
    
    allocations = []
    
    def backtrack(current, remaining, depth):
        if depth == n:
            if remaining == 0 and all(min_unit <= u <= max_unit for u in current):
                allocations.append([u * cfg['step'] for u in current])
            return
        
        start = max(min_unit, remaining - (n - depth - 1) * max_unit)
        end = min(max_unit, remaining - (n - depth - 1) * min_unit)
        
        for u in range(start, end + 1):
            backtrack(current + [u], remaining - u, depth + 1)
    
    backtrack([], total_units, 0)
    return allocations

def initialize_system():
    """预计算所有有效组合"""
    global valid_compositions, total_count
    validate_config()
    
    cfg = GLOBAL_CONFIG
    element_list = list(elements.keys())
    valid_compositions = []
    seen = set()
    
    # 生成元素组合
    for elements_group in tqdm(combinations(element_list, cfg['num_elements']), 
                              desc=f"预计算{cfg['num_elements']}元组合"):
        # 生成浓度分配
        for percents in generate_concentrations():
            # 标准化化学式
            sorted_group = sorted(elements_group, key=lambda x: elements_order.index(x))
            formula = "".join([f"{e}{p/100:.2f}" for e, p in zip(sorted_group, percents)])
            
            if formula not in seen:
                seen.add(formula)
                # 参数计算
                Omega, delta, VEC = calculate_parameters(elements_group, percents)
                if (Omega >= cfg['Omega_threshold'] and 
                    delta <= cfg['delta_threshold'] and 
                    VEC >= cfg['VEC_threshold']):
                    valid_compositions.append((elements_group, percents))
    
    total_count = len(valid_compositions)
    print(f"系统预计算完成，有效组合总数: {total_count}")

def calculate_parameters(selected, percents):
    """参数计算函数"""
    X = [p/100 for p in percents]
    Tm = [elements[e]['Tm'] for e in selected]
    radius = [elements[e]['radius'] for e in selected]
    VEC = [elements[e]['VEC'] for e in selected]
    
    # 加权平均熔点
    T_m = sum(t * x for t, x in zip(Tm, X))
    
    # 混合熵
    R = 8.314
    S_mix = -R * sum(x * np.log(x) for x in X if x > 0)
    
    # 混合焓
    H_mix = 0
    for i, j in combinations(range(len(selected)), 2):
        pair = "".join(sorted([selected[i], selected[j]]))
        H_mix += 4 * mixen_dict[pair] * X[i] * X[j]         # i=1，j>i时，乘以 4 的原因：2（对称性） × 2（权重因子）
    # 单位换算：ΔH_mix 从 kJ/mol 转换为 J/mol
    H_mix *= 1000
    
    # 计算 Ω
    Omega = (T_m * S_mix) / abs(H_mix) if H_mix != 0 else float('inf')

    # 计算 δ
    r_avg = sum(r * x for r, x in zip(radius, X))
    delta = np.sqrt(sum(x * (1 - r/r_avg)**2 for x, r in zip(X, radius))) * 100
    
    # 计算 VEC
    vec = sum(v * x for v, x in zip(VEC, X))
    
    return Omega, delta, vec

def generate_samples(n):
    """生成样本"""
    global total_count
    if total_count == 0:
        raise ValueError("没有有效组合可用，请检查筛选条件")
    
    if n > total_count:
        print(f"警告: 要求数量({n})超过有效组合数({total_count})，已自动调整")
        n = total_count
    
    samples = random.sample(valid_compositions, n)
    df = pd.DataFrame([format_sample(s) for s in samples])
    return df[['Co','Cr','Fe','Mn','Ni','V','formula','Omega','delta_r','VEC']]   #修改表头

def format_sample(sample):
    """格式化单个样本"""
    elements_group, percents = sample
    sorted_group = sorted(elements_group, key=lambda x: elements_order.index(x))
    formula = "".join([f"{e}{p/100:.2f}" for e, p in zip(sorted_group, percents)])
    
    sample_dict = {e: 0 for e in elements_order}
    for e, p in zip(elements_group, percents):
        sample_dict[e] = p
    
    Omega, delta, VEC = calculate_parameters(elements_group, percents)
    sample_dict.update({
        'formula': formula,
        'Omega': round(Omega, 2),
        'delta_r': round(delta, 2),
        'VEC': round(VEC, 2)
    })
    return sample_dict

# 使用示例
if __name__ == "__main__":

    # 全局配置参数
    GLOBAL_CONFIG = {
        'num_elements': 5,      # 主元数
        'min_percent': 16,       # 最小浓度（%）
        'max_percent': 24,      # 最大浓度（%）
        'step': 1,              # 浓度步长（%）
        'Omega_threshold': 1.1, # Ω阈值
        'delta_threshold': 6.6, # δ阈值
        'VEC_threshold': 8      # VEC阈值
    }
    
    initialize_system()
    result_df = generate_samples(50)
    print(result_df.to_string(index=False))
    print(f"\n成功生成 {len(result_df)} 个唯一样本")

预计算5元组合: 6it [00:01,  3.34it/s]

系统预计算完成，有效组合总数: 2175
 Co  Cr  Fe  Mn  Ni  V                        formula  Omega  delta_r  VEC
 22  21  21  17  19  0 Co0.22Cr0.21Fe0.21Mn0.17Ni0.19   5.97     3.36 8.01
 23  17  22  21  17  0 Co0.23Cr0.17Fe0.22Mn0.21Ni0.17   6.06     3.61 8.02
 23  20  19  19  19  0 Co0.23Cr0.20Fe0.19Mn0.19Ni0.19   5.79     3.49 8.02
 18  18  19  22  23  0 Co0.18Cr0.18Fe0.19Mn0.22Ni0.23   5.39     3.66 8.06
 21  16  21  18  24  0 Co0.21Cr0.16Fe0.21Mn0.18Ni0.24   5.59     3.46 8.19
 18  21  17  20  24  0 Co0.18Cr0.21Fe0.17Mn0.20Ni0.24   5.27     3.54 8.04
 20  18  23  20  19  0 Co0.20Cr0.18Fe0.23Mn0.20Ni0.19   6.02     3.54 8.02
 16  19  22  19  24  0 Co0.16Cr0.19Fe0.22Mn0.19Ni0.24   5.57     3.49 8.07
 21  18  21  21  19  0 Co0.21Cr0.18Fe0.21Mn0.21Ni0.19   5.86     3.60 8.02
 21  21  17  17  24  0 Co0.21Cr0.21Fe0.17Mn0.17Ni0.24   5.37     3.38 8.10
 19  21  17  21  22  0 Co0.19Cr0.21Fe0.17Mn0.21Ni0.22   5.40     3.59 8.00
 23  19  18  20  20  0 Co0.23Cr0.19Fe0.18Mn0.20Ni0.20   5.61     3.56 8.05
 22 

保存csv

In [5]:
result_df.to_csv('alloy_selected.csv', index=False)